<a href="https://colab.research.google.com/github/thispath98/NLP/blob/main/BERT/BERT_modoo_corpus_sentence_acceptability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[bert_naver_movie_colab_ipynb](https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=P58qy4--s5_x) 와 
[일상/연애 주제의 한국어 대화 'BERT'로 이진 분류 모델 만들기](https://velog.io/@seolini43/%EC%9D%BC%EC%83%81%EC%97%B0%EC%95%A0-%EC%A3%BC%EC%A0%9C%EC%9D%98-%ED%95%9C%EA%B5%AD%EC%96%B4-%EB%8C%80%ED%99%94-BERT%EB%A1%9C-%EC%9D%B4%EC%A7%84-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0%ED%8C%8C%EC%9D%B4%EC%8D%ACColab-%EC%BD%94%EB%93%9C) 를 보고 실습해보았습니다.


# Colab 환경 설정

먼저, GPU를 사용하기 위해 시스템 환경 유형 변경을 통해 `GPU`로 변경해주고, 아래 코드를 입력하여 라이브러리를 설치하고 import 해준다.


In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [3]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla K80


# 데이터셋 불러오기

## (1) 학습 데이터 불러오기

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


> 데이터 불러오고 쉐입을 확인한다.

In [5]:
import csv
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[말뭉치] 문장 문법성 판단/NIKL_CoLA_train.tsv',encoding="utf-8", delimiter='\t')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[말뭉치] 문장 문법성 판단/NIKL_CoLA_dev.tsv',encoding="utf-8", delimiter='\t')
df_train.shape  # (11823, 3)

(15876, 4)

In [6]:
df_train.sample(n=5)

,source,acceptability_label,source_annotation,sentence
4071,T02566,1,NaN,이때 많은 전류가 흐르지만 저항이 없으므로 에너지의 열 손실이 생기지 않는다.
2119,T01348,0,*,그는 세 사람이 다섯 사람이라고 말했다.
7057,T04404,1,NaN,요즘 대학생들은 외국에 살아 본 경험이 많이들 있거든요.
8917,T05580,0,*,"지난주에 누군가가 내 볼펜을 빌려갔는데, 그게 누가인지 생각이 안 나."
15573,T09815,0,*,열심히 영이는 철수가 논문을 썼다고 말했다.


In [7]:
df_train = df_train[['sentence', 'acceptability_label']]
df_train.head()

,sentence,acceptability_label
0,높은 달이 떴다.,1
1,달이 뜸이 높았다.,0
2,실없는 사람이 까불까불한다.,1
3,나는 철수에게 공을 던졌다.,1
4,내가 순이와 둘이서 다툰다.,1


# Train set 전처리
여기서부터 지금까지 전처리한 dataset이 BERT 모델의 입력데이터가 되기 위한 특수한 전처리가 필요하다.



## (1) [CLS]와 [SEP]
BERT 분류 모델의 경우 각 문장의 앞마다 [CLS]를 붙여 인식시키고, 문장의 종료는 [SEP]를 붙여 인식시킨다. [CLS]을 인식함으로써 문장의 처음이라 알 수 있게 하고, [SEP]을 인식함으로써 문장의 끝을 알 수 있게 하기 위함이다.

이전 글에서 BERT의 pretrain 방법 중 하나가 [SEP]를 인식하여 두 문장이 이어지는 문장인지, 관련 없는 문장인지 학습하는 것이었다. 아무튼 추가한 데이터 역시 학습을 해야하니 각 문장마다 [CLS]와 [SEP]를 붙여주어야 한다.

In [8]:
# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences_train = ["[CLS] " + str(s) + " [SEP]" for s in df_train['sentence']]
print(sentences_train[:5])

['[CLS] 높은 달이 떴다. [SEP]', '[CLS] 달이 뜸이 높았다. [SEP]', '[CLS] 실없는 사람이 까불까불한다. [SEP]', '[CLS] 나는 철수에게 공을 던졌다. [SEP]', '[CLS] 내가 순이와 둘이서 다툰다. [SEP]']


한편, '0'과 '1'의 라벨이 들어있는 컬럼을 'labels'이라는 array에 따로 저장한다.

In [9]:
labels_train = df_train['acceptability_label'].values
print(labels_train)

[1 0 1 ... 1 0 1]


## (2)서브워드 토크나이저 : WordPiece
BERT에서 사용하는 토크나이저인 WordPiece에 대해 지난번 글에서 설명했다. 단어를 토큰화 할 때, 단어집합에 없는 단어는 더 쪼개서 '##'을 붙여주는 방식이었다. BertTokenizer 라이브러리를 통해 이를 수행할 수 있는데, '안녕하세요!'라는 문장에 대해 적용해보았다.

현재 단어 집합이 비어있는 상태이므로 '안녕하세요'라는 단어가 단어집합에 없다. 따라서 이 단어는 한번 더 쪼개어 지는데 '##'을 붙임으로써 '##녕', '##하', '##세', '##요' 가 어떠한 단어의 서브워드라는 것을 나타내준다. 이렇게 나타내 줌으로써 단어의 의미를 잃지 않고 다시 원래 단어로 복원을 할 수 있게 된다.

그렇다면 이제 WordPiece tokenizer를 이용하여 전체 데이터에 토크나이징을 수행하도록 한다. 토크나이징을 하는 코드는 간단하다.

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts_train = [tokenizer.tokenize(s) for s in sentences_train]

In [11]:
print(sentences_train[0])  #토크나이징 전
print(tokenized_texts_train[0]) #토크나이징 후

[CLS] 높은 달이 떴다. [SEP]
['[CLS]', '높은', '달', '##이', '[UNK]', '.', '[SEP]']


이 이후에, 문장의 최대 시퀀스를 설정하여 정수 인코딩과 제로 패딩을 수행해준다.

In [12]:
MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids_train = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]

In [13]:
input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(input_ids_train[0])

[  101 55600  9061 10739   100   119   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


## (3) Attention Mask (어텐션 마스크)
어텐션 마스크에 대해선 이전 글에서 설명을 하지 않았는데, 어텐션 마스크란 0 값을 가지는 패딩 토큰에 대해서 어텐션 연산을 불필요하게 수행하지 않도록 단어와 패딩 토큰을 구분할 수 있게 알려주는 것을 말한다.

따라서 [40311, 9435, 102, 0, 0]와 같은 패딩된 데이터가 있을 때, 패딩된 값은 '0', 패딩되지 않은 단어는 '1'의 값을 갖도록 시리얼 데이터를 만들어 주어야 한다.

    예) 패딩된 데이터 = [40311, 9435, 102, 0, 0]
    >>어텐션 마스크 = [1.0, 1.0, 1.0, 0.0, 0.0]

In [14]:
attention_masks_train = []

for seq in input_ids_train:
    seq_mask = [float(i>0) for i in seq]
    attention_masks_train.append(seq_mask)

In [15]:
attention_masks_train[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

## (4) Train set, Validation set 설정
데이터를 모두 파이토치 텐서로 변환시킨다.

마지막으로 배치사이즈를 32로 설정하고, 입력데이터, 어텐션 마스크, 라벨을 하나의 데이터로 묶어 train_dataloader, validation_dataloader라는 입력데이터를 생성해준다.

In [19]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids_train,
                                                                                    labels_train, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks_train, 
                                                       input_ids_train,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  9747, 73894,  9638,  9739, 10622,  9642, 17706, 48549,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [25]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 4. Test set 전처리
Train set을 전처리 한 것과 같은 방식으로 Test set에도 같은 전처리 과정을 수행한다.


In [26]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in df_test['sentence']]

# 라벨 데이터
labels = df_test['acceptability_label'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 5. BERT 모델 불러오기
이제 BERT 모델을 불러오도록 한다. (여기서부터는 참조한 코드와 동일하다)

그 전에 GPU 설정을 위한 디바이스 설정을 해준다.

아래 코드를 실행하고 'There are 1 GPU(s) available.' 라는 문구가 출력되면 사용이 가능하다는 뜻이다.

In [27]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


이제 pretrain된 BERT 모델을 불러오도록 하겠다.

In [28]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

하이퍼 파라미터 값은 참조한 코드에 있는 값 그대로 설정했다. 에폭 수는 간단하게 4로 설정해주었다.

In [29]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


# 6. 모델 학습
필요한 함수를 설정해주고 난 뒤, 아래 코드를 실행하여 모델을 학습시켜준다.

훈련하는 데에는 `Tesla K80` GPU 기준으로 약 40분이 소요된다.

In [30]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [31]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.69
  Training epcoh took: 0:09:23

Running Validation...
  Accuracy: 0.49
  Validation took: 0:00:22

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.69
  Training epcoh took: 0:09:25

Running Validation...
  Accuracy: 0.55
  Validation took: 0:00:22

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.66
  Training epcoh took: 0:09:26

Running Validation...
  Accuracy: 0.53
  Validation took: 0:00:22

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.61
  Training epcoh took: 0:09:26

Running Validation...
  Accuracy: 0.53
  Validation took: 0:00:22

Training complete!


# 7. 테스트셋 평가
지금까지 pretrain된 BERT 모델에 추가로 9000개의 한글 대화 데이터를 추가로 학습시켰다. 이제 테스트 셋 데이터로 학습시킨 모델의 정확도를 측정해보도록 하겠다.

In [32]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.57
Test took: 0:00:28


# 8. 새로운 문장 테스트
이제 새로운 문장을 입력하여 분류가 잘 이루어지는지 살펴보도록 하자.

입력하는 문장도 BERT의 입력 형태로 만드는 작업이 필요하기 때문에 우선 아래 함수를 정의해준다.

In [33]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [34]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [37]:
test_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[말뭉치] 문장 문법성 판단/NIKL_CoLA_test.tsv',encoding="utf-8", delimiter='\t')
test_dataset = test_dataset[['sentence']]
test_dataset.head()

,sentence
0,나는 철수에게 공을 던져다 주었다.
1,먹은 것을 다 소화시켜야 한다.
2,그가 노래를 부르고는 내가 피아노를 쳤다.
3,철수가 영수의 손을 잡아서 눈물을 글썽거렸다.
4,그의 업적은 길이 빛난다.


In [94]:
ans = []
pred = []

for idx, sentences in enumerate(test_dataset['sentence']):
    logits = test_sentences([sentences])
    y_pred = '' if np.argmax(logits) == 1 else '문법 오류'
    print(f'{idx:<5,} : {sentences:^50} {y_pred}')

0     :                나는 철수에게 공을 던져다 주었다.                 문법 오류
1     :                 먹은 것을 다 소화시켜야 한다.                  문법 오류
2     :              그가 노래를 부르고는 내가 피아노를 쳤다.               문법 오류
3     :             철수가 영수의 손을 잡아서 눈물을 글썽거렸다.              문법 오류
4     :                   그의 업적은 길이 빛난다.                   문법 오류
5     :                     별이 반짝반짝한다.                     문법 오류
6     :                   영수와 철수가 같이하였다.                   문법 오류
7     :                화초가 시들었다. 꽃밭에 물을 줄까?                문법 오류
8     :                철호가 성실하다고 나에 의해 보인다.                문법 오류
9     :              노래는 순이가 부르고는 춤은 추지 않는다.               문법 오류
10    :                  철수가 제 방에 가서 잔다.                   문법 오류
11    :              성은 나서 날뛰는 사람은 물과 불을 모른다.              문법 오류
12    :             나는 범인이 검문소를 통과했다고 사실에 놀랐다.             문법 오류
13    :           철수가 영수를 만나서 영수에게 도와 달라고 부탁했다.            문법 오류
14    :                   나는 지금 밥을 먹겠다.                    문법 오류
15    :                  